In [ ]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

font_size = 18
ticks_size = 15

In [ ]:
file = open("mbpp_deepseek-coder-7b-instruct-v1.5_temp_1.0_final_outputs.pkl", 'rb')
original_data= pickle.load(file)
original_data = pd.DataFrame(original_data)
original_data

In [ ]:
data = original_data.copy()
data['ground_truth_test_1'] = ((data['result_unit_test_1'] == data['ground_truth_test_1']) |
                               (pd.isna(data['result_unit_test_1']) & pd.isna(data['ground_truth_test_1'])))
data['ground_truth_test_2'] = ((data['result_unit_test_2'] == data['ground_truth_test_2']) |
                               (pd.isna(data['result_unit_test_2']) & pd.isna(data['ground_truth_test_2'])))
data['ground_truth_test_3'] = ((data['result_unit_test_3'] == data['ground_truth_test_3']) |
                               (pd.isna(data['result_unit_test_3']) & pd.isna(data['ground_truth_test_3'])))

# Add critical errors

In [ ]:
data = data.fillna("None")

In [ ]:
all = True

if all:
    data_fixed = data
    original_list = list(range(0, 399, 1))
    result_list = [item for item in original_list for _ in range(200)]
    data_fixed['src_idx'] = result_list
    # Insert critical error when at least one of the ground truth tests is False
    data_fixed['critical'] = np.where((data_fixed['ground_truth_test_1'] == False) |
                                    (data_fixed['ground_truth_test_2'] == False) |
                                    (data_fixed['ground_truth_test_3'] == False), 1, 0)
    data_filtered = data_fixed
    result_list = [item for item in original_list for _ in range(200)]
    data_filtered['src_idx'] = result_list
else:
    data_fixed = data
    original_list = list(range(0, 399, 1))
    result_list = [item for item in original_list for _ in range(200)]
    data_fixed['src_idx'] = result_list
    data_fixed['critical'] = np.where(data_fixed['ground_truth_test_1'] == False, 1, 0)
    data_filtered = data_fixed
    result_list = [item for item in original_list for _ in range(200)]
    data_filtered['src_idx'] = result_list

In [ ]:
# run for all metrics below
metric = 'random'
metric = 'oracle'
metric = 'majority_voting'

n_samples=200
n_examples=399
aux_1 = np.arange(1, n_samples+1)
aux_2= np.tile(aux_1, n_examples)
data_filtered['n_sample'] = aux_2
critical_n_lbd = np.full((n_samples, n_examples), -1)

for K in range(1, n_samples+1):
    selected_df = data_filtered[data_filtered['n_sample']<K+1]
    if metric != 'random':
        if metric == 'oracle':
            chosen_df = selected_df.loc[selected_df.groupby(selected_df.index // n_samples )['critical'].idxmin()]
        if metric == 'majority_voting':
            indexes_to_keep = []
            for src_idx in range (0, n_examples):
                try: # this works if not all entries are failed
                    mode_output = selected_df[selected_df['src_idx'] == src_idx]['result_unit_test_1'].loc[lambda x: x.astype(str).apply(lambda entry: not entry.startswith("failed"))].value_counts().idxmax()
                except:
                    mode_output = selected_df[selected_df['src_idx']==src_idx]['result_unit_test_1'].value_counts().idxmax()
                selected_df_src = selected_df[selected_df['src_idx']==src_idx]
                indexes = selected_df_src['result_unit_test_1'].index
                for i in range(0,len(indexes)):
                    if selected_df_src['result_unit_test_1'].iloc[i] == mode_output:
                        break
                indexes_to_keep.append(indexes[i])
            chosen_df = selected_df.loc[indexes_to_keep]
    else:
        chosen_df = data_filtered[data_filtered['n_sample'] == K]
    critical_n_lbd[K-1] = np.array(chosen_df['critical'])

critical_absolute = critical_n_lbd.sum(axis=1)
critical_rate = critical_n_lbd.sum(axis=1)/n_examples

if metric == 'oracle':
    critical_absolute_oracle = critical_absolute
    critical_rate_oracle = critical_rate
if metric == 'majority_voting':
    critical_absolute_maj = critical_absolute
    critical_rate_maj = critical_rate
if metric == 'random':
    critical_absolute_random = critical_absolute
    critical_rate_random = critical_rate

In [ ]:
def rerankers(data, n_samples, n_examples, metric, set):
    aux_1 = np.arange(1, n_samples+1)
    aux_2= np.tile(aux_1, n_examples)
    data['n_sample'] = aux_2
    critical_n_lbd = np.full((n_samples, n_examples), -1)

    for K in range(1, n_samples+1):
        selected_df = data[data['n_sample']<K+1]
        if metric != 'random':
            if metric == 'oracle':
                chosen_df = selected_df.loc[selected_df.groupby(selected_df.index // n_samples )['critical'].idxmin()]
            if metric == 'majority_voting':
                indexes_to_keep = []
                starting_at = 0
                if set=='test':
                    starting_at=199
                for src_idx in range (starting_at, starting_at + n_examples):
                    try: # this works if not all entries are failed
                        mode_output = selected_df[selected_df['src_idx'] == src_idx]['result_unit_test_1'].loc[lambda x: x.astype(str).apply(lambda entry: not entry.startswith("failed"))].value_counts().idxmax()
                    except:
                        mode_output = selected_df[selected_df['src_idx']==src_idx]['result_unit_test_1'].value_counts().idxmax()
                    selected_df_src = selected_df[selected_df['src_idx']==src_idx]
                    indexes = selected_df_src['result_unit_test_1'].index
                    for i in range(0,len(indexes)):
                        if selected_df_src['result_unit_test_1'].iloc[i] == mode_output:
                            break
                    indexes_to_keep.append(indexes[i])
                chosen_df = selected_df.loc[indexes_to_keep]
        else:
            chosen_df = data[data['n_sample'] == K]
        critical_n_lbd[K-1] = np.array(chosen_df['critical'])

    critical_absolute = critical_n_lbd.sum(axis=1)
    critical_rate = critical_n_lbd.sum(axis=1)/n_examples
    
    return critical_absolute, critical_rate



In [ ]:
n_samples=200
n_examples=int(399/2)
half_length = int(n_samples * n_examples)
data_filtered_dev = data_filtered[:half_length]
data_filtered_test = data_filtered[half_length:]

metric = 'oracle'
critical_absolute_oracle_dev, critical_rate_oracle_dev = rerankers(data_filtered_dev,  n_samples, n_examples, metric, 'dev')
critical_absolute_oracle_test, critical_rate_oracle_test = rerankers(data_filtered_test,  n_samples, n_examples+1, metric, 'test')

metric = 'majority_voting'
critical_absolute_maj_dev, critical_rate_maj_dev = rerankers(data_filtered_dev,  n_samples, n_examples, metric, 'dev')
critical_absolute_maj_test, critical_rate_maj_test = rerankers(data_filtered_test,  n_samples, n_examples+1, metric, 'test')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))  # 1 row, 2 columns

# Plot 1: critical errors
axes[0].plot(critical_rate_oracle_dev, label="oracle dev", color='blue')
axes[0].plot(critical_rate_maj_dev, label="maj dev", color='red')
axes[0].plot(critical_rate_oracle_test, label="oracle test", color='blue', linestyle='dashed')
axes[0].plot(critical_rate_maj_test, label="maj test", color='red', linestyle='dashed')
axes[0].set_xlabel("N", fontsize=font_size)
axes[0].set_ylabel("% critical errors", fontsize=font_size)
axes[0].set_title("% critical errors in the set", fontsize=font_size)
axes[0].tick_params(axis='both', labelsize=ticks_size)

# Plot 2: log critical errors
axes[1].plot(np.log(critical_rate_oracle_dev)-np.log(critical_rate_oracle_dev)[0], label="oracle dev", color='blue')
axes[1].plot(np.log(critical_rate_maj_dev)-np.log(critical_rate_maj_dev)[0], label="maj dev", color='red')
axes[1].plot(np.log(critical_rate_oracle_test)-np.log(critical_rate_oracle_test)[0], label="oracle test", color='blue', linestyle='dashed')
axes[1].plot(np.log(critical_rate_maj_test)-np.log(critical_rate_maj_test)[0], label="maj test", color='red', linestyle='dashed')
axes[1].set_xlabel("N", fontsize=font_size)
axes[1].set_ylabel("log critical errors", fontsize=font_size)
axes[1].set_title("log critical errors in the set", fontsize=font_size)
axes[1].tick_params(axis='both', labelsize=ticks_size)


axes[0].legend(loc='lower center', bbox_to_anchor=(1.1, -0.4), fontsize=font_size, frameon=False, ncol=4)

plt.show()

# Fitting with least squares

In [ ]:
from scipy.optimize import least_squares
import math
from entmax import sparsemax, entmax15, entmax_bisect
import scipy.integrate as integrate
from scipy.special import gammaln

import scipy.integrate as integrate
from scipy.special import comb
from scipy.special import logsumexp
import torch

In [ ]:
def log_beta_distribution(alpha, beta):
    beta_lognorm = -gammaln(alpha+beta) + gammaln(alpha) + gammaln(beta)
    return lambda tau: (alpha-1)*np.log(tau) + (beta-1)*np.log(1-tau) - beta_lognorm

def integrate_expression(tau_start, tau_end):
    result, _ = integrate.quad(lambda tau: expression(tau, N), tau_start, tau_end)
    return result

def fit_alpha_beta(p, n_samples_fit, y):
    # p[0]= alpha, p[1]=beta
    # y: ground truth

    q=0
    N = np.arange(1, n_samples_fit+1)
    comet_log_failure_rate = np.zeros_like(N, dtype=float)

    for i, n in enumerate(N):
        somation = 0
        for K in range(n+1):
            i_values_1 = np.arange(1, K + 1)
            i_values_2 = np.arange(1, n - K + 1)
            i_values_3 = np.arange(1, n+1)
            t_values = np.arange(n-K+1, n+1)
            j_values = np.arange(1, n+1)
            somation_log =  np.log(comb(n, K)) + np.sum(np.log(p[0] + i_values_1 - 1)) + np.sum(np.log(p[1] + i_values_2 - 1)) - np.sum(np.log(p[0] + p[1] + i_values_3 - 1))
            somation += np.exp(somation_log)*np.sum(q**(t_values-1)) / np.sum(q**(j_values-1))
        result = somation
        comet_log_failure_rate[i] = np.log(result)

    eps = p[0]/(p[0]+p[1]) # eps = alpha/(alpha+beta)
    expression_comet = comet_log_failure_rate - np.log(eps)
    return expression_comet - y


def fit_q_entmaxalpha(p, n_samples_fit, fitted_alpha, fitted_beta, y):

    N = np.arange(1, n_samples_fit+1)
    log_failure_rate = np.zeros_like(N, dtype=float)

    for i, n in enumerate(N):
        somation = 0
        for K in range(n+1):
            i_values_1 = np.arange(1, K + 1)
            i_values_2 = np.arange(1, n - K + 1)
            i_values_3 = np.arange(1, n+1)
            t_values = np.arange(n-K+1, n+1)
            j_values = np.arange(1, n+1)            
            x = torch.tensor((N[:n]) * np.log(p[0]))
            results_entmax = []
            expression_values_entmax = entmax_bisect(x, alpha=p[1]).tolist()
            results_entmax.append(expression_values_entmax)
            results_entmax = np.array(results_entmax)
            somation_log =  np.log(comb(n, K)) + np.sum(np.log(fitted_alpha + i_values_1 - 1)) + np.sum(np.log(fitted_beta + i_values_2 - 1)) - np.sum(np.log(fitted_alpha + fitted_beta + i_values_3 - 1)) #+ np.log(np.sum(q**(t_values-1))) - np.log(np.sum(q**(j_values-1)))
            somation += np.exp(somation_log) * results_entmax[0][n-K:n].sum()
        result = somation
        log_failure_rate[i] = np.log(result)
    
    eps = fitted_alpha/(fitted_alpha+fitted_beta) # eps = alpha/(alpha+beta)
    expression_cometkiwi = log_failure_rate - np.log(eps)
    
    return expression_cometkiwi - y

In [ ]:
n_samples_fit = int(5/5 * n_samples)

In [ ]:
y_comet = (np.log(critical_rate_oracle_dev[:n_samples_fit]) - np.log(critical_rate_oracle_dev[0]))[:n_samples_fit]
y_mbr = (np.log(critical_rate_maj_dev[:n_samples_fit]) - np.log(critical_rate_maj_dev[0]))[:n_samples_fit]

p0 = np.zeros((2,))
p0[0] = 1.0
p0[1] = 1.0

first_fit = least_squares(fit_alpha_beta, p0, loss='soft_l1', f_scale=1.0, args=(n_samples_fit, y_comet), max_nfev=1000, bounds=([0.1,0.1],[1000,1000]))

p1 = np.zeros((2,))
p1[0] = 0.1
p1[1] = 0.75
second_fit_mbr = least_squares(fit_q_entmaxalpha, p1, loss='soft_l1', f_scale=1.0, args=(n_samples_fit, first_fit.x[0], first_fit.x[1], y_mbr), max_nfev=100, bounds=([0.001, 0.001],[1, 1]))

# Plots

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.figure(figsize=(8, 5))
labels = ['majority voting', 'oracle']

alpha = first_fit.x[0]
beta = first_fit.x[1]
eps = alpha / (alpha + beta)
N = np.arange(1, n_samples+1)
fitted_q_mbr = second_fit_mbr.x[0]
entmax_alpha_mbr = second_fit_mbr.x[1]
fitted_qs = [fitted_q_mbr]
entmax_alphas = [entmax_alpha_mbr]

for fitted_q, entmax_alpha, color_reranker in zip(fitted_qs, entmax_alphas, ['red']):
    q_values = [fitted_q, 0]
    n_points = n_samples

    for q, color, label in zip(q_values, [colors[3], colors[4]], labels):
        
        log_failure_rate = np.zeros_like(N, dtype=float)
        
        for i, n in enumerate(N):
            somation = 0
            for K in range(n+1):
                i_values_1 = np.arange(1, K + 1)
                i_values_2 = np.arange(1, n - K + 1)
                i_values_3 = np.arange(1, n+1)
                t_values = np.arange(n-K+1, n+1)
                j_values = np.arange(1, n+1)
                if q!= 0:
                    x = torch.tensor((N[:n]) * np.log(q))
                    results_entmax = []
                    expression_values_entmax = entmax_bisect(x, alpha=entmax_alpha).tolist()
                    results_entmax.append(expression_values_entmax)
                    results_entmax = np.array(results_entmax)
                    somation_log =  np.log(comb(n, K)) + np.sum(np.log(alpha + i_values_1 - 1)) + np.sum(np.log(beta + i_values_2 - 1)) - np.sum(np.log(alpha + beta + i_values_3 - 1))
                    somation += np.exp(somation_log) * results_entmax[0][n-K:n].sum()
                else:
                    somation_log =  np.log(comb(n, K)) + np.sum(np.log(alpha + i_values_1 - 1)) + np.sum(np.log(beta + i_values_2 - 1)) - np.sum(np.log(alpha + beta + i_values_3 - 1))
                    somation += np.exp(somation_log)*np.sum(q**(t_values-1)) / np.sum(q**(j_values-1))

            result = somation
            log_failure_rate[i] = np.log(result)

        if q!= 0:
            plt.plot(N, log_failure_rate - np.log(eps), color=color, linewidth=4.0, linestyle='solid', label=label)
        else:
            plt.plot(N, log_failure_rate - np.log(eps), color=color, linewidth=4.0, linestyle='dashed', label=label)


plt.scatter(N, np.log(critical_rate_maj_test) - np.log(critical_rate_maj_test[0]), color=colors[3], marker='o', s=.5 * (plt.rcParams['lines.markersize'] ** 2))
plt.scatter(N, np.log(critical_rate_oracle_test[:n_points]) - np.log(critical_rate_oracle_test[0]), color=colors[4], marker='o', s=.5 * (plt.rcParams['lines.markersize'] ** 2))
plt.xlabel('$N$', fontsize=font_size)
plt.ylabel('Log failure rate (wrt baseline)', fontsize=font_size)
plt.legend(loc='upper right', fontsize=font_size, ncol=1)
plt.yticks([-1.5, -1.0, -0.5, 0], fontsize=font_size)
plt.xticks([0, 50, 100, 150, 200], fontsize=font_size)
plt.tick_params(axis='both', labelsize=ticks_size)

plt.savefig('temp1.0_alltests_codegen_apred%.4f_bpred%.4f_qpred%.4f_entpred%.4f_.pdf' % (alpha,beta,fitted_q,entmax_alpha), bbox_inches='tight')
plt.show()


In [ ]:
print(fitted_q_mbr, entmax_alpha_mbr)
print(alpha, beta)

In [ ]:
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

plt.figure(figsize=(8, 5))
labels = ['majority voting', 'oracle']
eps = alpha / (alpha + beta)
N = np.arange(1, n_samples+1)
fitted_qs = [fitted_q_mbr]
entmax_alphas = [entmax_alpha_mbr]

for fitted_q, entmax_alpha, color_reranker in zip(fitted_qs, entmax_alphas, ['red']):
    q_values = [fitted_q, 0]
    n_points = n_samples

    for q, color, label in zip(q_values, [colors[3], colors[4]], labels):
        
        log_failure_rate = np.zeros_like(N, dtype=float)
        
        for i, n in enumerate(N):
            somation = 0
            for K in range(n+1):
                i_values_1 = np.arange(1, K + 1)
                i_values_2 = np.arange(1, n - K + 1)
                i_values_3 = np.arange(1, n+1)
                t_values = np.arange(n-K+1, n+1)
                j_values = np.arange(1, n+1)
                if q!= 0:
                    x = torch.tensor((N[:n]) * np.log(q))
                    results_entmax = []
                    expression_values_entmax = entmax_bisect(x, alpha=entmax_alpha).tolist()
                    results_entmax.append(expression_values_entmax)
                    results_entmax = np.array(results_entmax)
                    somation_log =  np.log(comb(n, K)) + np.sum(np.log(alpha + i_values_1 - 1)) + np.sum(np.log(beta + i_values_2 - 1)) - np.sum(np.log(alpha + beta + i_values_3 - 1)) #+ np.log(np.sum(q**(t_values-1))) - np.log(np.sum(q**(j_values-1)))
                    somation += np.exp(somation_log) * results_entmax[0][n-K:n].sum()
                else:
                    somation_log =  np.log(comb(n, K)) + np.sum(np.log(alpha + i_values_1 - 1)) + np.sum(np.log(beta + i_values_2 - 1)) - np.sum(np.log(alpha + beta + i_values_3 - 1))
                    somation += np.exp(somation_log)*np.sum(q**(t_values-1)) / np.sum(q**(j_values-1))
            result = somation
            log_failure_rate[i] = np.log(result)

        if q!= 0:
            plt.plot(N, log_failure_rate - np.log(eps), color=color, linewidth=4.0, linestyle='solid', label=label)
        else:
            plt.plot(N, log_failure_rate - np.log(eps), color=color, linewidth=4.0, linestyle='dashed', label=label)

plt.scatter(N, np.log(critical_rate_maj_dev) - np.log(critical_rate_maj_dev[0]), color=colors[3], marker='o', s=.5 * (plt.rcParams['lines.markersize'] ** 2))
plt.scatter(N, np.log(critical_rate_oracle_dev[:n_points]) - np.log(critical_rate_oracle_dev[0]), color=colors[4], marker='o', s=.5 * (plt.rcParams['lines.markersize'] ** 2))
plt.xlabel('$N$', fontsize=font_size)
plt.ylabel('Log failure rate (wrt baseline)', fontsize=font_size)
plt.legend(loc='upper right', fontsize=font_size, ncol=1)
plt.yticks([-1.5, -1.0, -0.5, 0], fontsize=font_size)
plt.xticks([0, 50, 100, 150, 200], fontsize=font_size)
plt.tick_params(axis='both', labelsize=ticks_size)

plt.savefig('temp1.0_alltests_codegen_dev_apred%.4f_bpred%.4f_qpred%.4f_entpred%.4f_.pdf' % (alpha,beta,fitted_q,entmax_alpha), bbox_inches='tight')
plt.show()
